In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import gensim.downloader as api
import pandas as pd
from torch import flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
# import torch.nn.functional as F

In [56]:
train_data_0_path = "/home3/luharj/DLNLP/Assignment1/data/train/ClassificationDataset-train0.xlsx"
valid_data_0_path = "/home3/luharj/DLNLP/Assignment1/data/valid/ClassificationDataset-valid0.xlsx"

In [57]:
glove_model = api.load("glove-wiki-gigaword-300")

In [58]:
train_df_0 = pd.read_excel(train_data_0_path)
valid_df_0 = pd.read_excel(valid_data_0_path)

In [59]:
class CNN(nn.Module) :
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3,300), padding=0, stride=1)
        self.conv2 = nn.Conv2d(1, 16, kernel_size=(4,300), padding=0, stride=1)
        self.conv3 = nn.Conv2d(1, 16, kernel_size=(5,300), padding=0, stride=1)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=(68), stride=2)
        self.pool2 = nn.MaxPool1d(kernel_size=(67), stride=2)
        self.pool3 = nn.MaxPool1d(kernel_size=(66), stride=2)
        self.fc1 = nn.Linear(in_features=16*3,out_features=16)
        self.fc2 = nn.Linear(in_features=16,out_features=8)
        self.fc3 = nn.Linear(in_features=8,out_features=3)
        self.softmax = nn.Softmax(dim = 1)
    def forward(self, x) :
        x1 = self.relu(self.conv1(x))
        x1 = x1.squeeze()
        x1 = self.pool1(x1)
        x1 = flatten(x1, 1)
        
        x2 = self.relu(self.conv2(x))
        x2 = x2.squeeze()
        x2 = self.pool2(x2)
        x2 = flatten(x2, 1)
        
        x3 = self.relu(self.conv3(x))
        x3 = x3.squeeze()
        x3 = self.pool3(x3)
        x3 = flatten(x3, 1)
        
        x = torch.cat((x1, x2, x3), dim=1)
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [60]:
def get_labels(word) :
    if word == "negative" :
        return 0
    if word == "neutral" :
        return 1
    else:
        return 2

In [61]:
def get_embedding(sentences) :
    words = sentences.split(" ")
    embeddings = []
    for word in words:
        if word in glove_model:
            embeddings.append(glove_model[word])
        else :
            embeddings.append(np.random.uniform(-1,1,300))
    return np.array(embeddings)

In [62]:
def reshaped_data(data) :
    desired_shape = (70,300)
    rows_to_copy = min(len(data), 70)
    padded_data = np.zeros(desired_shape)
    padded_data[:rows_to_copy, :] = data[:rows_to_copy, :]
    return np.array(padded_data)

In [63]:
def get_data(data_frame) :
    training_data = []
    training_labels = []
    for index, row in data_frame.iterrows() :
        data = get_embedding(row[1])
        labels = get_labels(row[0])
        training_data.append(reshaped_data(data))
        training_labels.append(labels)
    return np.array(training_data), np.array(training_labels)

In [64]:
training_data, training_labels = get_data(data_frame=train_df_0)
validation_data, validation_labels = get_data(data_frame=valid_df_0)

In [65]:
X = torch.Tensor(training_data)
y = torch.LongTensor(training_labels)

In [66]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
X_train = X
y_train = y
X_val = torch.Tensor(validation_data)
y_val = torch.LongTensor(validation_labels)

In [68]:
X_train = X_train.unsqueeze(1)
X_val = X_val.unsqueeze(1)

In [69]:
print(X_train.shape, X_val.shape)

torch.Size([2798, 1, 70, 300]) torch.Size([1203, 1, 70, 300])


In [70]:
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [71]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [72]:
epochs = 20  # You can adjust the number of epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 300), stride=(1, 1))
  (conv2): Conv2d(1, 16, kernel_size=(4, 300), stride=(1, 1))
  (conv3): Conv2d(1, 16, kernel_size=(5, 300), stride=(1, 1))
  (relu): ReLU()
  (pool1): MaxPool1d(kernel_size=68, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool1d(kernel_size=67, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool1d(kernel_size=66, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=48, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In [73]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    true_labels = []
    predicted_labels = []
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        output_labels = torch.argmax(outputs, dim=1)
        
        # Append true and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(output_labels.cpu().numpy())

        running_loss += loss.item()

    accuracy = accuracy_score(true_labels, predicted_labels)
    
    # Calculate macro and micro F1 scores for the epoch
    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
    micro_f1 = f1_score(true_labels, predicted_labels, average='micro')
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}, Accuracy: {accuracy}, Macro F1: {macro_f1}, Micro F1: {micro_f1}")

Epoch 1/20, Loss: 0.9653283818201586, Accuracy: 0.6054324517512509, Macro F1: 0.2566209001852566, Micro F1: 0.6054324517512509
Epoch 2/20, Loss: 0.8518708524378863, Accuracy: 0.7022873481057899, Macro F1: 0.3902433006035568, Micro F1: 0.7022873481057899
Epoch 3/20, Loss: 0.7995099696246061, Accuracy: 0.7494639027877055, Macro F1: 0.45932550435773195, Micro F1: 0.7494639027877055
Epoch 4/20, Loss: 0.7332020618698813, Accuracy: 0.82987848463188, Macro F1: 0.5401056660757325, Micro F1: 0.82987848463188


Epoch 5/20, Loss: 0.6637921536510641, Accuracy: 0.8995711222301644, Macro F1: 0.6001626960783106, Micro F1: 0.8995711222301644
Epoch 6/20, Loss: 0.622836249795827, Accuracy: 0.9342387419585418, Macro F1: 0.6289709490488528, Micro F1: 0.9342387419585418
Epoch 7/20, Loss: 0.6074977029453624, Accuracy: 0.9481772694781987, Macro F1: 0.6395717235409965, Micro F1: 0.9481772694781987
Epoch 8/20, Loss: 0.5985553847117857, Accuracy: 0.9538956397426733, Macro F1: 0.643520408412482, Micro F1: 0.9538956397426733
Epoch 9/20, Loss: 0.5948514640331268, Accuracy: 0.956397426733381, Macro F1: 0.6458258900920654, Micro F1: 0.9563974267333809
Epoch 10/20, Loss: 0.593230664730072, Accuracy: 0.9578270192994996, Macro F1: 0.6459654829673805, Micro F1: 0.9578270192994996
Epoch 11/20, Loss: 0.5919714989987287, Accuracy: 0.9592566118656183, Macro F1: 0.647782565181657, Micro F1: 0.9592566118656183
Epoch 12/20, Loss: 0.5903777805241671, Accuracy: 0.9606862044317369, Macro F1: 0.6481110490408158, Micro F1: 0.960

In [74]:
model.eval()

CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 300), stride=(1, 1))
  (conv2): Conv2d(1, 16, kernel_size=(4, 300), stride=(1, 1))
  (conv3): Conv2d(1, 16, kernel_size=(5, 300), stride=(1, 1))
  (relu): ReLU()
  (pool1): MaxPool1d(kernel_size=68, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool1d(kernel_size=67, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool1d(kernel_size=66, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=48, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In [75]:
true_labels = []
predicted_labels = []

In [76]:
for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(inputs)
    output_labels = torch.argmax(outputs, dim = 1)
    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(output_labels.cpu().numpy())

In [77]:
test_accuracy = accuracy_score(true_labels, predicted_labels)

In [78]:
test_macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
test_micro_f1 = f1_score(true_labels, predicted_labels, average='micro')

In [79]:
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Macro F1: {test_macro_f1}")
print(f"Test Micro F1: {test_micro_f1}")

Test Accuracy: 0.7522859517871987
Test Macro F1: 0.4840490653786936
Test Micro F1: 0.7522859517871987


In [84]:
model1 = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
epochs = 20  # You can adjust the number of epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)

CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 300), stride=(1, 1))
  (conv2): Conv2d(1, 16, kernel_size=(4, 300), stride=(1, 1))
  (conv3): Conv2d(1, 16, kernel_size=(5, 300), stride=(1, 1))
  (relu): ReLU()
  (pool1): MaxPool1d(kernel_size=68, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool1d(kernel_size=67, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool1d(kernel_size=66, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=48, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In [85]:
# Training and validation loop
for epoch in range(epochs):
    model1.train()
    running_loss = 0.0
    true_labels = []
    predicted_labels = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        output_labels = torch.argmax(outputs, dim=1)

        # Append true and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(output_labels.cpu().numpy())

        running_loss += loss.item()

    # Calculate accuracy, macro F1, and micro F1 on the training set
    accuracy = accuracy_score(true_labels, predicted_labels)
    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
    micro_f1 = f1_score(true_labels, predicted_labels, average='micro')

    # Initialize variables for validation
    val_true_labels = []
    val_predicted_labels = []
    val_running_loss = 0.0

    model1.eval()  # Switch to evaluation mode for validation
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

            val_outputs = model1(val_inputs)
            val_loss = criterion(val_outputs, val_labels)

            val_output_labels = torch.argmax(val_outputs, dim=1)

            # Append true and predicted labels for validation
            val_true_labels.extend(val_labels.cpu().numpy())
            val_predicted_labels.extend(val_output_labels.cpu().numpy())

            val_running_loss += val_loss.item()

    # Calculate accuracy, macro F1, and micro F1 on the validation set
    val_accuracy = accuracy_score(val_true_labels, val_predicted_labels)
    val_macro_f1 = f1_score(val_true_labels, val_predicted_labels, average='macro')
    val_micro_f1 = f1_score(val_true_labels, val_predicted_labels, average='micro')

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {running_loss / len(train_loader)}, Accuracy: {accuracy}, Macro F1: {macro_f1}, Micro F1: {micro_f1}, Validation Loss: {val_running_loss / len(val_loader)}, Val Accuracy: {val_accuracy}, Val Macro F1: {val_macro_f1}, Val Micro F1: {val_micro_f1}")

Epoch 1/20, Train Loss: 0.951683139259165, Accuracy: 0.5843459614010007, Macro F1: 0.31163488603598416, Micro F1: 0.5843459614010007, Validation Loss: 0.897390604019165, Val Accuracy: 0.6467165419783873, Val Macro F1: 0.2618206293117954, Val Micro F1: 0.6467165419783873
Epoch 2/20, Train Loss: 0.8667425391348925, Accuracy: 0.6833452466047176, Macro F1: 0.3529755251680385, Micro F1: 0.6833452466047176, Validation Loss: 0.84684010242161, Val Accuracy: 0.7024106400665004, Val Macro F1: 0.3990565673747444, Val Micro F1: 0.7024106400665004
Epoch 3/20, Train Loss: 0.8162185603922064, Accuracy: 0.7333809864188706, Macro F1: 0.43920746213719486, Micro F1: 0.7333809864188705, Validation Loss: 0.8286621727441487, Val Accuracy: 0.7273482959268496, Val Macro F1: 0.44742022975034, Val Micro F1: 0.7273482959268495
Epoch 4/20, Train Loss: 0.7510879608717832, Accuracy: 0.8102215868477484, Macro F1: 0.5215786433752071, Micro F1: 0.8102215868477485, Validation Loss: 0.8030459943570589, Val Accuracy: 0.7